## Pre-training custom classifier using SST-5

Reference: [ShannonAI Github](https://github.com/ShannonAI/Self_Explaining_Structures_Improve_NLP_Models)  

What used?
  * BertTokenizer
  * BertForSequenceClassification
 
You can download SST-5 dataset from above reference. Please check "Prepare Datasets and Models" section.

Dataset consists of 3 files, "train.txt, dev.txt, test.txt". I use train.txt and dev.txt as training set, and test.txt as validation set.

Custom classifier is MLP with 1 hidden layer. In "Save model parameters" section, entire model parameters are saved. If you want to use this pre-trained result with different custom layer, yo have to modify your classifier in this file and "Bert-custom_linear.ipynb" simultaneously.

### Requirement
  * transformers == 4.6.0
  * pytorch == 1.8.0
  * numpy == 1.1.92
  * pandas == 1.2.3
  * tqdm == 4.60.0
  * scikit-learn == 0.24.1

<a id="Table"></a>
### Table  
  - [Import library](#Import)
  - [Check train set](#Trainset)
  - [Tokenizer, Encoding data](#Encode)
  - [Define custom classifier model](#Classifier)
  - [Call custom classifier model, Data loaders](#Model)
    - Can modify batch_size in this block
  - [Optimizer & Scheduler](#Optim)
    - Can modify epochs, lr, etc. in this block
  - [Running model](#Run)
  - [Save model parameters](#Save)

<a id="Import"></a>
### Import library
  - [Return to table](#Table)

In [2]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch import nn, optim
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertForMaskedLM, BertModel
from transformers import AdamW, get_linear_schedule_with_warmup

In [3]:
USE_CUDA = torch.cuda.is_available()
print('Use', 'cuda' if USE_CUDA else 'cpu')
device = torch.device('cuda' if USE_CUDA else 'cpu')
#device = torch.device('cpu')

Use cuda


<a id="Trainset"></a>
### Check train set
  - [Return to table](#Table)

In [21]:
train_df = pd.read_csv("./SST_modified/train.txt", sep="\t", names=['label', 'Sentence'], header=None)
dev = pd.read_csv("./SST_modified/dev.txt", sep="\t", names=['label', 'Sentence'], header=None)
test = pd.read_csv("./SST_modified/test.txt", sep="\t", names=['label', 'Sentence'], header=None)
#train_df = pd.read_csv('./data/aug_train_final.csv')

In [22]:
train_df = pd.concat([train_df, dev],  ignore_index=True)

Label distribution


In [23]:
train_df['label'].value_counts()

3    2601
1    2507
2    1853
4    1453
0    1231
Name: label, dtype: int64

<a id="Encode"></a>
### Tokenizer, Encoding data
  - [Return to table](#Table)

In [24]:
#Use below tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

#Encode train sentence
encoded_data_train = tokenizer.batch_encode_plus(
    train_df.Sentence.values, #Sentence data
    add_special_tokens=True,    #Encoded with special tokens relative to their model
    return_attention_mask=True, #Return attention mask according to tokenizer defined by max_length att.
    padding='longest',           #Padding!
    #truncation=True,            
    return_tensors='pt'         #Return torch tensor
)

#Encode validation sentence
encoded_data_val = tokenizer.batch_encode_plus(
    test.Sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest', 
    #truncation=True, 
    return_tensors='pt'
)

In [26]:
"""
encoded_data_train has 3 keys:
  *input_ids
  *token_type_ids
  *attention_mask
"""

#Input
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_df.label.values)

#Validation
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(test.label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

<a id="Classifier"></a>
### Define custom classifier model
  - [Return to table](#Table)

In [27]:
class Classifier(nn.Module):
    def __init__(self, n_classes):
        super(Classifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop1 = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size//2)
        self.output = nn.Linear(self.bert.config.hidden_size//2,  n_classes)
        
    def forward(self, input_ids, attention_mask):
        x = self.bert(
                    input_ids=input_ids,
                    attention_mask=attention_mask)
        x = self.drop1(x[1])
        x = F.relu(self.fc1(x))
        x = F.softmax(self.output(x), dim=1)
        return x

<a id="Model"></a>
### Call custom classifier model, Data loaders
  - [Return to table](#Table)

In [29]:
model = Classifier(len(train_df.label.unique()))
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Classifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [30]:
batch_size = 16

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

<a id="Optim"></a>
### Optimizer & Scheduler
  - [Return to table](#Table)

In [33]:
optimizer = AdamW(model.parameters(),
                  lr=2e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

loss_fn = nn.CrossEntropyLoss().to(device)

<a id="Run"></a>
### Running model
  - [Return to table](#Table)

In [32]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    predictions = []
    
    correct_predictions = 0
    losses = []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        _, preds = torch.max(outputs, dim=1)
        predictions.append(preds)
        
        loss = loss_fn(outputs, batch[2])
        
        correct_predictions += torch.sum(preds == batch[2])
        losses.append(loss.item())
        
    return correct_predictions.double() / len(dataloader_validation.dataset), np.mean(losses), predictions

In [35]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    correct_predictions = 0
    losses = []


    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:


        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }       

        outputs = model(**inputs)
        
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, batch[2])
        
        correct_predictions += torch.sum(preds == batch[2])
        
        losses.append(loss.item())
        
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
        optimizer.zero_grad()
         
    #If you want to save model parameters, please modify below code.
    #Line 80 save model for every epochs
    #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = np.mean(losses)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_acc, val_loss, _ = evaluate(dataloader_validation)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'Valdation accuracy: {val_acc}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/603 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.4727733006722497
Validation loss: 1.404926652530972
Valdation accuracy: 0.4995475113122172


Epoch 2:   0%|          | 0/603 [00:00<?, ?it/s]


Epoch 2
Training loss: 1.3756605463834544
Validation loss: 1.4008609107930026
Valdation accuracy: 0.497737556561086


...


Epoch 3
Training loss: 1.3212854397632985
Validation loss: 1.3861511791352745
Valdation accuracy: 0.5190045248868779


Epoch 4:   0%|          | 0/603 [00:00<?, ?it/s]


Epoch 4
Training loss: 1.2715625142181295
Validation loss: 1.3901194325453943
Valdation accuracy: 0.5108597285067874


Epoch 5:   0%|          | 0/603 [00:00<?, ?it/s]


Epoch 5
Training loss: 1.2377172241954266
Validation loss: 1.3869493513656177
Valdation accuracy: 0.5113122171945702


<a id="Save"></a>
### Save model parameters
  - [Return to table](#Table)

In [36]:
torch.save(model.state_dict(), f'model_save/SST_5_fine_tuning.model')